In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import classification_report

In [3]:
crypto_btc = yf.download(tickers='BTC-USD', start='2017-12-01', end='2022-12-31', interval='1d').drop(columns=['Adj Close']).round(2)

[*********************100%***********************]  1 of 1 completed


In [4]:
crypto_btc['Percent Change'] = crypto_btc['Close'].pct_change()
crypto_btc = crypto_btc.dropna()

In [6]:
crypto_btc['ema_9'] = crypto_btc['Close'].ewm(9).mean()

In [5]:
crypto_btc['sma_5'] = crypto_btc['Close'].rolling(5).mean()

In [15]:
crypto_btc['sma_10'] = crypto_btc['Close'].rolling(10).mean()

In [16]:
crypto_btc['sma_15'] = crypto_btc['Close'].rolling(10).mean()

In [17]:
crypto_btc['sma_30'] = crypto_btc['Close'].rolling(10).mean()

In [18]:
crypto_btc = crypto_btc.dropna()
crypto_btc

,Open,High,Low,Close,Volume,Percent Change,sma_5,ema_9,sma_10,sma_15,sma_30
Date,,,,,,,,,,,
2017-12-15,16601.30,18154.10,16601.30,17706.90,14309999616,0.068999,17006.260,15805.773708,16442.550,16442.550,16442.550
2017-12-16,17760.30,19716.70,17515.30,19497.40,12740599808,0.101119,17518.380,16270.650306,16963.140,16963.140,16963.140
2017-12-17,19475.80,20089.00,18974.10,19140.80,13314599936,-0.018290,17863.460,16622.946465,17087.250,17087.250,17087.250
2017-12-18,19106.40,19371.00,18355.90,19114.20,14839499776,-0.001390,18404.660,16921.934616,17341.730,17341.730,17341.730
2017-12-19,19118.30,19177.80,17275.40,17776.70,16894499840,-0.069974,18647.200,17022.506447,17601.580,17601.580,17601.580
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,16842.25,16920.12,16812.37,16919.80,11886957804,0.004620,16847.368,16929.242883,16795.343,16795.343,16795.343
2022-12-27,16919.29,16959.85,16642.07,16717.17,15748580239,-0.011976,16824.734,16908.035594,16787.551,16787.551,16787.551
2022-12-28,16716.40,16768.17,16497.56,16552.57,17005713920,-0.009846,16775.858,16872.489035,16767.010,16767.010,16767.010


In [19]:
X = crypto_btc[['Open', 'High', 'Low', 'Volume', 'Percent Change', 'sma_5', 'sma_10', 'sma_15', 'sma_30', 'ema_9']].copy()
display(X.head())

,Open,High,Low,Volume,Percent Change,sma_5,sma_10,sma_15,sma_30,ema_9
Date,,,,,,,,,,
2017-12-15,16601.3,18154.1,16601.3,14309999616,0.068999,17006.26,16442.55,16442.55,16442.55,15805.773708
2017-12-16,17760.3,19716.7,17515.3,12740599808,0.101119,17518.38,16963.14,16963.14,16963.14,16270.650306
2017-12-17,19475.8,20089.0,18974.1,13314599936,-0.018290,17863.46,17087.25,17087.25,17087.25,16622.946465
2017-12-18,19106.4,19371.0,18355.9,14839499776,-0.001390,18404.66,17341.73,17341.73,17341.73,16921.934616
2017-12-19,19118.3,19177.8,17275.4,16894499840,-0.069974,18647.20,17601.58,17601.58,17601.58,17022.506447


In [20]:
y = crypto_btc['Close']

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)
kr = KernelRidge(kernel='linear', alpha=5000)

In [34]:
kr.fit(X_train, y_train)

c:\Users\Kyle Hagan\anaconda3\envs\project1\lib\site-packages\sklearn\linear_model\_ridge.py:197: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  "Singular matrix in solving dual problem. Using "


KernelRidge(alpha=5000)

In [35]:
predict = kr.predict(X_train)

In [36]:
crypto_btc_pred = crypto_btc.copy().drop(columns=['Open','Volume','High','Low','Percent Change','sma_5', 'sma_10', 'sma_15', 'sma_30', 'ema_9']).rename(columns={'Close':'Actual Price'})
crypto_btc_pred

crypto_btc_pred['Predicted Price'] = predict
#crypto_btc_pred['Price Accuracy']= (crypto_btc_pred['Predicted Price']/crypto_btc_pred['Actual Price']-1)*100

ValueError: Length of values (1381) does not match length of index (1842)

In [65]:
#crypto_btc['EMA_9'] = crypto_btc['Close'].ewm(9).mean().shift()
#crypto_btc['SMA_5'] = crypto_btc['Close'].rolling(5).mean().shift()
#crypto_btc['SMA_10'] = crypto_btc['Close'].rolling(10).mean().shift()
#crypto_btc['SMA_15'] = crypto_btc['Close'].rolling(15).mean().shift()
#crypto_btc['SMA_30'] = crypto_btc['Close'].rolling(30).mean().shift()
#crypto_btc['Percent Change'] = crypto_btc['Close'].pct_change()

In [66]:
#crypto_btc = crypto_btc.dropna()
#crypto_btc

In [67]:
crypto_btc.reset_index(inplace=True)
fig = px.line(crypto_btc, x='Date', y='Close', title='Close Price with Range Slider')
fig.update_layout(hovermode="x")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=2, label="2y", step="year", stepmode="backward"),
            dict(step="all")
            
        ])
    )
)
fig.show()

In [68]:
'''
df = ()
def kernel_predict(df_pred):
    X = df.drop(columns=['Date'])
    y = df[['Close']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.7, random_state=42)
    if df is crypto_btc:
        kr = KernelRidge(kernel='linear', alpha=5000)
    else:
        kr = KernelRidge(kernel='polynomial', degree=1, gamma=.05)
    kr.fit(X_train, y_train)
    predict = kr.predict(X)
    if df is crypto_btc:
        df_pred = df.copy().drop(columns=['Open','Volume']).rename(columns={'Close':'Actual Price'})
    else:
        df_pred = df.copy().drop(columns=['Open']).rename(columns={'Close':'Actual Price'})
    df_pred['Predicted Price'] = predict
    df_pred['Price Accuracy']= (df_pred['Predicted Price']/df_pred['Actual Price']-1)*100
    return df_pred
'''

"\ndf = ()\ndef kernel_predict(df_pred):\n    X = df.drop(columns=['Date'])\n    y = df[['Close']]\n    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.7, random_state=42)\n    if df is crypto_btc:\n        kr = KernelRidge(kernel='linear', alpha=5000)\n    else:\n        kr = KernelRidge(kernel='polynomial', degree=1, gamma=.05)\n    kr.fit(X_train, y_train)\n    predict = kr.predict(X)\n    if df is crypto_btc:\n        df_pred = df.copy().drop(columns=['Open','Volume']).rename(columns={'Close':'Actual Price'})\n    else:\n        df_pred = df.copy().drop(columns=['Open']).rename(columns={'Close':'Actual Price'})\n    df_pred['Predicted Price'] = predict\n    df_pred['Price Accuracy']= (df_pred['Predicted Price']/df_pred['Actual Price']-1)*100\n    return df_pred\n"

In [69]:
X = crypto_btc[['ema_9','sma_5']].shift().dropna().copy()

In [74]:
crypto_btc

,Open,High,Low,Close,Volume,ema_9,sma_5,pct change
Date,,,,,,,,
2017-12-05,11685.70,12032.00,11604.60,11916.70,6895260160,11441.587888,11389.460,0.022261
2017-12-06,11923.40,14369.10,11923.40,14291.50,12656300032,12049.816972,12052.640,0.199283
2017-12-07,14266.10,17899.70,14057.30,17899.70,17950699520,13171.121988,13417.660,0.252472
2017-12-08,17802.90,18353.40,14336.90,16569.40,21135998976,13767.800190,14466.900,-0.074320
2017-12-09,16523.30,16783.00,13674.90,15178.20,13911300096,13998.039658,15171.100,-0.083962
...,...,...,...,...,...,...,...,...
2022-12-26,16842.25,16920.12,16812.37,16919.80,11886957804,16929.242883,16847.368,0.004620
2022-12-27,16919.29,16959.85,16642.07,16717.17,15748580239,16908.035594,16824.734,-0.011976
2022-12-28,16716.40,16768.17,16497.56,16552.57,17005713920,16872.489035,16775.858,-0.009846


In [ ]:
X = crypto_btc.drop(columns=['Date'])
y = crypto_btc[['Close']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.7, random_state=42)
kr = KernelRidge(kernel='linear', alpha=5000)
kr.fit(X_train, y_train)
predict = kr.predict(crypto_btc.drop(columns=['Date']))
crypto_btc_pred = crypto_btc.copy().drop(columns=['Open','Volume']).rename(columns={'Close':'Actual Price'})
crypto_btc_pred['Predicted Price'] = predict
crypto_btc_pred['Price Accuracy']= (crypto_btc_pred['Predicted Price']/crypto_btc_pred['Actual Price']-1)*100
    

In [ ]:
btc_mae, btc_r_square, btc_r_square= (),(),()

btc_mae = mean_absolute_error(y['Close'], predict)
btc_rmse = np.sqrt(mean_squared_error(y['Close'], predict))
btc_r_square = r2_score(y['Close'], predict)
print(btc_mae)
print(btc_rmse)
print(btc_r_square)

In [ ]:
crypto_btc_pred

In [ ]:
crypto_btc_pred['Price Accuracy'].describe()

In [ ]:
crypto_btc_pred.hvplot(
    x='Date',
    y=[('Actual Price') ,'Predicted Price'],
    grid=True
)

In [ ]:
print(f"BTC Kernel Ridge RMSE is {btc_rmse.round(2)}")

In [ ]:
future_df = pd.DataFrame()
  

future_df['Date'] = pd.date_range('2018-02-01', periods = 1826, freq ='1D')
future_df[['Close','Open','Volume']]= (0)

future_df


In [ ]:
crypto_btc
X = crypto_btc.drop(columns=['Date'])
y = crypto_btc[['Close']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.7, random_state=42)
kr = KernelRidge(kernel='linear', alpha=5000)
kr.fit(X_train, y_train)
predict = kr.predict(crypto_btc.drop(columns=['Date']))
btc_future = future_df.copy().drop(columns=['Open','Volume']).rename(columns={'Close':'Actual Price'})
btc_future['Predicted Price'] = predict

btc_future 

In [ ]:
btc_mae, btc_r_square, btc_r_square= (),(),()

btc_mae = mean_absolute_error(y['Close'], predict)
btc_rmse = np.sqrt(mean_squared_error(y['Close'], predict))
btc_r_square = r2_score(y['Close'], predict)
print(btc_mae)
print(btc_rmse)
print(btc_r_square)

In [ ]:
crypto_btc2 = yf.download(tickers='BTC-USD', start='2018-01-01', end='2023-01-30').drop(columns=['Adj Close']).round(2).reset_index()
crypto_btc2['EMA_9'] = crypto_btc2['Close'].ewm(9).mean().shift()
crypto_btc2['SMA_5'] = crypto_btc2['Close'].rolling(5).mean().shift()
crypto_btc2['SMA_10'] = crypto_btc2['Close'].rolling(10).mean().shift()
crypto_btc2['SMA_15'] = crypto_btc2['Close'].rolling(15).mean().shift()
crypto_btc2['SMA_30'] = crypto_btc2['Close'].rolling(30).mean().shift()
crypto_btc2['Percent Change'] = crypto_btc2['Close'].pct_change()
#crypto_btc2 = crypto_btc2.dropna()
crypto_btc2
btc_future['Actual Price'] = crypto_btc2['Close'].round(0)
btc_future['Price Accuracy']= (btc_future['Predicted Price']/btc_future['Actual Price']-1)*100
btc_future.set_index('Date').tail(30)

In [ ]:
display(btc_future.loc[btc_future['Date'] == '2020-01-27'])
display(btc_future.loc[btc_future['Date'] == '2020-02-27'])

In [ ]:
from bokeh.models.formatters import BasicTickFormatter

btc_future.hvplot(
    x='Date',
    y=[('Actual Price') ,'Predicted Price'],
    grid=True
)